In [1]:
import pandas as pd
import numpy as np
from utils_optimized import *
import warnings
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LinearRegression
warnings.simplefilter('ignore')
total_splits = 4

In [2]:
train, test = load_data()

In [3]:
def aggregate_new(df, take_values=False):
    mode = lambda x: stats.mode(x).mode[0]
    fst = lambda vec: vec.iloc[0]
    simple_trend = lambda vec: np.sum(vec.shift(1)-vec[1:])
   
    num_features = ['min', 'max', 'median', 'sum', simple_trend]
    cat_features = [unique_cnt, 'min', 'max', mode]
   
    #'first_prch_num', 'q', 'v_l', 'month', 'time_weight', 'v_l_tw'
    res = df.groupby('id')[['q', 'v_l']].agg({
#         'first_prch_num':'max',
        'q':['sum', 'mean'],
        'v_l':'sum',
#         'month':unique_cnt,
#         'time_weight':['min', 'median'],
#         'v_l_tw':'median'
    })
 
    if take_values:
        return res.values, res.index
    else:
        return res

In [4]:
agg_func = aggregate_new

In [ ]:
for offset in range(total_splits):
    print(offset)
    X_train, y_train = calculate_target(train, offset=offset)
    X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train)
    print("Adding features..")
    X_tr, X_val = add_features(X_tr, X_val, sort=True)
    print("Aggregating..")
    X_tr, X_val = agg_func(X_tr, take_values=False), agg_func(X_val, take_values=False)
    
    save_split('data/holdouts.hdf', X_tr, X_val, y_tr, y_val, str(offset))

0
Adding features..


### DataFrame with models results

In [ ]:
lgb = LGBMClassifier()
scores_lgb, probas_lgb = cross_val(lgb, None, None, return_proba=True, splits=total_splits,
                           splits_file='data/holdouts.hdf', verbose=False)
print(np.mean(scores_lgb))

In [ ]:
cb = CatBoostClassifier()
cat_features=[]
scores_cb, probas_cb = cross_val(cb, None, None, return_proba=True, splits=total_splits,
                           splits_file='data/holdouts.hdf', verbose=True,
                          cat_features=cat_features)
print(np.mean(scores_cb))

In [ ]:
xgb = XGBClassifier()
scores_xgb, probas_xgb = cross_val(xgb, None, None, return_proba=True, splits=5,
                                   splits_file='data/holdouts.hdf', verbose=False)
print(np.mean(scores_xgb))

Add columns: xgb, KNN etc.

In [ ]:
X = pd.DataFrame({'cb': pd.concat(probas_cb), 'lgb': pd.concat(probas_lgb)})

In [ ]:
target = []
for offset in range(total_splits):
    X_tr, X_val, y_tr, y_val = load_split('data/holdouts.hdf', offset)
    target.append(y_val)
target = pd.concat(target)

# Stacking

In [ ]:
for col in X_val.columns:
    print(col,"-",roc_auc_score(y_val, X_val[col]))

## LGB

In [ ]:
from sklearn.model_selection import train_test_split as tr_val

In [ ]:
X_tr, X_val, y_tr, y_val = tr_val(X, target, train_size=0.75, shuffle=True)

In [ ]:
y_pred = lgb.fit(X_tr, y_tr).predict_proba(X_val)[:, 1]
roc_auc_score(y_val, y_pred)

In [ ]:
roc_auc_score(y_val, X_val.iloc[:, 0]), roc_auc_score(y_val, X_val.iloc[:, 1])

## Linear

In [ ]:
lr = LinearRegression(fit_intercept=False).fit(X_tr, y_tr)

In [ ]:
y_pred = lr.fit(X_tr, y_tr).predict(X_val)
roc_auc_score(y_val, y_pred)

# Submission

Set whichever stacker you chose

In [ ]:
clf = LinearRegression(fit_intercept=False)

In [ ]:
clf.fit(X, target)

In [25]:
###SOME MAGIC WITH GENERATING SUBMISSIONS FROM DIFFERENT MODELS
# suppose, now we have submission_cb, submission_lgb, submission_xgb

In [ ]:
X_test_probas = pd.DataFrame({'cb': submission_cb.proba, 'lgb': submission_lgb.proba})

In [ ]:
sample_submission = pd.read_csv('data/sample_submission.csv')
sample_submission.proba = clf.predict(X_test_probas)